# Preflop Heads-Up Explorer

Analyse hero performance in true heads-up lineups (only one opponent seated). Use this notebook to profile profitability by position, street progression, and preflop line selection.

> Heads-up hands are filtered via `seat_count_start = 2` in the warehouse tables.

## Study Goals

- Quantify overall winrate and bb/100 in heads-up pots.
- Break down profit, VPIP/PFR, and showdown rates by position (BTN vs BB).
- Track results by deepest street reached (flop, turn, river).
- Compare profitability across preflop line families (open raise, 3-bet, call vs open, walks, etc.).
- Provide scaffolding for deeper slices such as stack depth, villain aggression, and board texture buckets.

In [1]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    PROJECT_ROOT / "data" / "warehouse" / "drivehud.sqlite",
    PROJECT_ROOT / "data" / "warehouse" / "ignition.sqlite",
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_DIR = PROJECT_ROOT / "analysis" / "cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

In [2]:
import sys

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import sqlite3
from typing import Sequence

import numpy as np
import pandas as pd
from IPython.display import display

from analysis.sqlite_utils import connect_readonly

In [3]:
# --- Configuration ---
FORCE_RELOAD = False
HEADS_UP_ONLY = True  # Toggle to reuse the loader for short-handed slices later
MIN_HAND_SAMPLE = 25  # Warn when aggregates are based on very small samples

CACHE_PATH = CACHE_DIR / "preflop_heads_up_hands.parquet"

In [4]:
try:
    import pyarrow  # type: ignore
    _PARQUET_SUPPORTED = True
except ImportError:  # Optional dependency; fall back to fastparquet
    try:
        import fastparquet  # type: ignore
        _PARQUET_SUPPORTED = True
    except ImportError:
        _PARQUET_SUPPORTED = False

PARQUET_SUPPORTED = _PARQUET_SUPPORTED
del _PARQUET_SUPPORTED

if not PARQUET_SUPPORTED:
    print("Parquet dependencies (pyarrow/fastparquet) are unavailable; caching will be skipped.")


Parquet dependencies (pyarrow/fastparquet) are unavailable; caching will be skipped.


In [5]:
STREET_ORDER = pd.CategoricalDtype(["Preflop", "Flop", "Turn", "River"], ordered=True)
PREFLOP_LINE_ORDER = [
    "Open Raise",
    "ISO Raise",
    "3-Bet",
    "4-Bet+",
    "Call vs Open",
    "Limp/Complete",
    "BB Walk / Fold",
    "BTN Folded",
    "Uncategorised",
]
FLAG_COLUMNS = [
    "vpip", "pfr", "three_bet", "four_bet", "call_open", "cold_call", "iso_raise",
    "squeeze", "steal_attempt", "steal_success", "faced_3b", "fold_to_3b", "three_bet_vs_steal",
]

def assign_preflop_line(row: pd.Series) -> str:
    if not row.get("vpip", 0):
        return "BB Walk / Fold" if row.get("position_pre") == "BB" else "BTN Folded"
    if row.get("four_bet"):
        return "4-Bet+"
    if row.get("three_bet"):
        return "3-Bet"
    if row.get("pfr"):
        return "ISO Raise" if row.get("iso_raise") else "Open Raise"
    if row.get("call_open") or row.get("cold_call"):
        return "Call vs Open"
    if row.get("vpip"):
        return "Limp/Complete"
    return "Uncategorised"

def _bb_per_100(series: pd.Series) -> float:
    hands = len(series)
    if hands == 0:
        return float("nan")
    values = series.fillna(0.0)
    return 100.0 * float(values.sum()) / hands

def summarise_groups(frame: pd.DataFrame, group_cols: Sequence[str]) -> pd.DataFrame:
    if frame.empty:
        return pd.DataFrame()
    grouped = (
        frame.groupby(list(group_cols), dropna=False)
        .agg(
            hands=("hand_id", "nunique"),
            net_bb=("net_bb", lambda s: float(s.fillna(0.0).sum())),
            net_dollars=("net_dollars", lambda s: float(s.sum())),
            bb_per_100=("net_bb", _bb_per_100),
            vpip_pct=("vpip", "mean"),
            pfr_pct=("pfr", "mean"),
            three_bet_pct=("three_bet", "mean"),
            call_open_pct=("call_open", "mean"),
            showdown_pct=("showdown", "mean"),
            went_to_flop=("saw_flop", "mean"),
            went_to_turn=("saw_turn", "mean"),
            went_to_river=("saw_river", "mean"),
            avg_pot_bb=("total_pot_bb", "mean"),
        )
        .sort_values("net_bb", ascending=False)
    )
    pct_cols = [
        "vpip_pct", "pfr_pct", "three_bet_pct", "call_open_pct", "showdown_pct",
        "went_to_flop", "went_to_turn", "went_to_river",
    ]
    grouped[pct_cols] = grouped[pct_cols] * 100.0
    return grouped.round({
        "net_bb": 1,
        "net_dollars": 2,
        "bb_per_100": 2,
        "avg_pot_bb": 2,
        "vpip_pct": 1,
        "pfr_pct": 1,
        "three_bet_pct": 1,
        "call_open_pct": 1,
        "showdown_pct": 1,
        "went_to_flop": 1,
        "went_to_turn": 1,
        "went_to_river": 1,
    })

def load_heads_up_hero_hands(
    db_path: Path,
    cache_path: Path | None = None,
    force: bool = False,
    heads_up_only: bool = True,
) -> pd.DataFrame:
    cache_active = bool(cache_path) and PARQUET_SUPPORTED
    if cache_path and cache_path.exists() and not force:
        if cache_active:
            try:
                return pd.read_parquet(cache_path)
            except Exception:
                print(f"Failed to load cache at {cache_path}; refreshing from database.")
        else:
            print("Parquet support missing; ignoring existing cache file.")

    where_clause = "WHERE 1=1"
    if heads_up_only:
        where_clause += " AND h.seat_count_start = 2"
    query = f"""
    SELECT
        h.hand_id,
        h.started_at_local,
        h.started_at_utc,
        h.seat_count_start,
        h.total_pot_c,
        h.rake_c,
        s.position_pre,
        s.role_pre,
        r.net_c,
        r.won_c,
        r.showdown,
        r.hand_class,
        COALESCE(hb.bb_c, 0) AS bb_c,
        CASE WHEN h.board_flop IS NOT NULL THEN 1 ELSE 0 END AS saw_flop,
        CASE WHEN h.board_turn IS NOT NULL THEN 1 ELSE 0 END AS saw_turn,
        CASE WHEN h.board_river IS NOT NULL THEN 1 ELSE 0 END AS saw_river,
        CASE
            WHEN h.board_river IS NOT NULL THEN 'River'
            WHEN h.board_turn IS NOT NULL THEN 'Turn'
            WHEN h.board_flop IS NOT NULL THEN 'Flop'
            ELSE 'Preflop'
        END AS deepest_street,
        v.vpip,
        v.pfr,
        v.three_bet,
        v.four_bet,
        v.call_open,
        v.cold_call,
        v.iso_raise,
        v.squeeze,
        v.steal_attempt,
        v.steal_success,
        v.faced_3b,
        v.fold_to_3b,
        v.three_bet_vs_steal
    FROM hands h
    JOIN seats s ON s.hand_id = h.hand_id AND s.is_hero = 1
    JOIN results r ON r.hand_id = h.hand_id AND r.seat_no = s.seat_no
    LEFT JOIN v_hand_bb hb ON hb.hand_id = h.hand_id
    LEFT JOIN v_hero_preflop_ext v ON v.hand_id = h.hand_id
    {where_clause}
    ORDER BY h.started_at_utc
    """
    with connect_readonly(db_path) as conn:
        frame = pd.read_sql_query(query, conn)

    for col in ("started_at_local", "started_at_utc"):
        if col in frame.columns:
            frame[col] = pd.to_datetime(frame[col], errors="coerce")

    for col in FLAG_COLUMNS:
        if col not in frame.columns:
            frame[col] = 0
        frame[col] = frame[col].fillna(0).astype(int)

    frame["bb_c"] = frame["bb_c"].astype(float)
    frame["net_dollars"] = frame["net_c"] / 100.0
    frame["won_dollars"] = frame["won_c"] / 100.0
    frame["net_bb"] = np.where(frame["bb_c"] > 0, frame["net_c"] / frame["bb_c"], np.nan)
    frame["won_bb"] = np.where(frame["bb_c"] > 0, frame["won_c"] / frame["bb_c"], np.nan)
    frame["total_pot_bb"] = np.where(frame["bb_c"] > 0, frame["total_pot_c"] / frame["bb_c"], np.nan)
    frame["rake_bb"] = np.where(frame["bb_c"] > 0, frame["rake_c"] / frame["bb_c"], np.nan)

    bool_cols = ["showdown", "saw_flop", "saw_turn", "saw_river"]
    for col in bool_cols:
        frame[col] = frame[col].astype(bool)

    frame["deepest_street"] = pd.Categorical(frame["deepest_street"], dtype=STREET_ORDER)
    frame["preflop_line"] = frame.apply(assign_preflop_line, axis=1)
    frame["preflop_line"] = pd.Categorical(frame["preflop_line"], categories=PREFLOP_LINE_ORDER, ordered=True)

    if cache_active:
        frame.to_parquet(cache_path, index=False)
    elif cache_path and not PARQUET_SUPPORTED:
        print("Skipped writing parquet cache because pyarrow/fastparquet is not installed.")
    return frame


In [6]:
heads_up_df = load_heads_up_hero_hands(
    DB_PATH,
    cache_path=CACHE_PATH,
    force=FORCE_RELOAD,
    heads_up_only=HEADS_UP_ONLY,
)

print(f"Loaded {len(heads_up_df):,} heads-up hero hands from {DB_PATH.name}.")
if not heads_up_df.empty:
    display(heads_up_df.head())

Skipped writing parquet cache because pyarrow/fastparquet is not installed.
Loaded 527 heads-up hero hands from drivehud.sqlite.


,hand_id,started_at_local,started_at_utc,seat_count_start,total_pot_c,rake_c,position_pre,role_pre,net_c,won_c,...,faced_3b,fold_to_3b,three_bet_vs_steal,net_dollars,won_dollars,net_bb,won_bb,total_pot_bb,rake_bb,preflop_line
0,4816844224,2025-08-21 05:14:55+10:00,2025-08-20 19:14:55+00:00,2,20,1,BTN,BTN,9,19,...,0,0,0,0.09,0.19,0.9,1.9,2.0,0.1,Limp/Complete
1,4816844408,2025-08-21 05:15:35+10:00,2025-08-20 19:15:35+00:00,2,15,0,SB,SB,5,15,...,0,0,0,0.05,0.15,0.5,1.5,1.5,0.0,BTN Folded
2,4816844477,2025-08-21 05:15:47+10:00,2025-08-20 19:15:47+00:00,2,20,0,BTN,BTN,10,20,...,0,0,0,0.10,0.20,1.0,2.0,2.0,0.0,Open Raise
3,4816844556,2025-08-21 05:16:04+10:00,2025-08-20 19:16:04+00:00,2,172,8,SB,SB,78,164,...,0,0,0,0.78,1.64,7.8,16.4,17.2,0.8,Call vs Open
4,4816844855,2025-08-21 05:17:05+10:00,2025-08-20 19:17:05+00:00,2,40,2,BTN,BTN,-20,0,...,0,0,0,-0.20,0.00,-2.0,0.0,4.0,0.2,Limp/Complete


In [7]:
if heads_up_df.empty:
    print("No heads-up hands available for analysis.")
else:
    total_hands = len(heads_up_df)
    summary_rows = [
        ("Hands", total_hands),
        ("Net (bb)", round(heads_up_df["net_bb"].fillna(0).sum(), 1)),
        ("Net ($)", round(heads_up_df["net_dollars"].sum(), 2)),
        ("bb/100", round(_bb_per_100(heads_up_df["net_bb"]), 2)),
        ("VPIP %", round(heads_up_df["vpip"].mean() * 100, 1)),
        ("PFR %", round(heads_up_df["pfr"].mean() * 100, 1)),
        ("3-Bet %", round(heads_up_df["three_bet"].mean() * 100, 1)),
        ("Showdown %", round(heads_up_df["showdown"].mean() * 100, 1)),
        ("Avg Pot (bb)", round(heads_up_df["total_pot_bb"].mean(), 2)),
    ]
    overall_summary = pd.DataFrame(summary_rows, columns=["Metric", "Value"])
    display(overall_summary)
    if total_hands < MIN_HAND_SAMPLE:
        print(f"Warning: only {total_hands} hands — treat all per-100 stats with caution.")

,Metric,Value
0,Hands,527.00
1,Net (bb),-135.40
2,Net ($),-13.54
3,bb/100,-25.69
4,VPIP %,51.00
5,PFR %,32.40
6,3-Bet %,4.40
7,Showdown %,0.00
8,Avg Pot (bb),7.18


In [8]:
if heads_up_df.empty:
    print("No position breakdown available.")
else:
    position_summary = summarise_groups(heads_up_df, ["position_pre"])
    display(position_summary)

,hands,net_bb,net_dollars,bb_per_100,vpip_pct,pfr_pct,three_bet_pct,call_open_pct,showdown_pct,went_to_flop,went_to_turn,went_to_river,avg_pot_bb
position_pre,,,,,,,,,,,,,
BTN,261,-20.8,-2.08,-7.97,67.0,50.6,0.0,0.4,0.0,40.2,24.5,15.7,6.10
SB,266,-114.6,-11.46,-43.08,35.3,14.7,8.6,19.5,0.0,37.2,25.6,18.4,8.24


In [9]:
if heads_up_df.empty:
    print("No street progression breakdown available.")
else:
    street_summary = summarise_groups(heads_up_df, ["deepest_street"])
    ordered = street_summary.reindex(STREET_ORDER.categories, fill_value=np.nan)
    display(ordered)

/tmp/ipykernel_2260/3257466663.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frame.groupby(list(group_cols), dropna=False)


,hands,net_bb,net_dollars,bb_per_100,vpip_pct,pfr_pct,three_bet_pct,call_open_pct,showdown_pct,went_to_flop,went_to_turn,went_to_river,avg_pot_bb
Preflop,323,15.7,1.57,4.86,27.2,25.1,4.3,0.0,0.0,0.0,0.0,0.0,2.02
Flop,72,38.1,3.81,52.92,87.5,47.2,6.9,20.8,0.0,100.0,0.0,0.0,7.21
Turn,42,17.9,1.79,42.62,88.1,40.5,2.4,26.2,0.0,100.0,100.0,0.0,8.49
River,90,-207.1,-20.71,-230.11,90.0,43.3,3.3,30.0,0.0,100.0,100.0,100.0,25.07


In [10]:
if heads_up_df.empty:
    print("No preflop line breakdown available.")
else:
    line_summary = summarise_groups(heads_up_df, ["preflop_line"])
    display(line_summary)

/tmp/ipykernel_2260/3257466663.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frame.groupby(list(group_cols), dropna=False)


,hands,net_bb,net_dollars,bb_per_100,vpip_pct,pfr_pct,three_bet_pct,call_open_pct,showdown_pct,went_to_flop,went_to_turn,went_to_river,avg_pot_bb
preflop_line,,,,,,,,,,,,,
3-Bet,23,79.6,7.96,346.09,100.0,100.0,100.0,0.0,0.0,39.1,17.4,13.0,17.40
Open Raise,132,38.1,3.81,28.86,100.0,100.0,0.0,0.8,0.0,52.3,31.8,20.5,9.73
4-Bet+,0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Uncategorised,0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BB Walk / Fold,0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ISO Raise,16,-38.8,-3.88,-242.50,100.0,100.0,0.0,0.0,0.0,75.0,62.5,56.2,21.80
Call vs Open,52,-57.1,-5.71,-109.81,100.0,0.0,0.0,100.0,0.0,100.0,71.2,50.0,17.49
Limp/Complete,46,-61.2,-6.12,-133.04,100.0,0.0,0.0,0.0,0.0,84.8,54.3,34.8,6.07
BTN Folded,258,-96.0,-9.60,-37.21,0.0,0.0,0.0,0.0,0.0,8.9,5.4,3.5,2.18


In [11]:
if heads_up_df.empty:
    print("No combined breakdown available.")
else:
    line_position = summarise_groups(heads_up_df, ["position_pre", "preflop_line"])
    display(line_position)

    line_street = summarise_groups(heads_up_df, ["preflop_line", "deepest_street"])
    display(line_street)

/tmp/ipykernel_2260/3257466663.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frame.groupby(list(group_cols), dropna=False)


hands  net_bb  net_dollars  bb_per_100  vpip_pct  \
position_pre preflop_line                                                       
SB           3-Bet              23    79.6         7.96      346.09     100.0   
BTN          Open Raise        132    38.1         3.81       28.86     100.0   
             Limp/Complete      43   -21.4        -2.14      -49.77     100.0   
             BTN Folded         86   -37.5        -3.75      -43.60       0.0   
SB           ISO Raise          16   -38.8        -3.88     -242.50     100.0   
             Limp/Complete       3   -39.8        -3.98    -1326.67     100.0   
             Call vs Open       52   -57.1        -5.71     -109.81     100.0   
             BTN Folded        172   -58.5        -5.85      -34.01       0.0   
BTN          ISO Raise           0     NaN          NaN         NaN       NaN   
             3-Bet               0     NaN          NaN         NaN       NaN   
             4-Bet+              0     NaN          NaN         NaN       NaN   
             Call vs Open        0     NaN          NaN         NaN       NaN   
             BB Walk / Fold      0     NaN          NaN         NaN       NaN   
             Uncategorised       0     NaN          NaN         NaN       NaN   
SB           Open Raise          0     NaN          NaN         NaN       NaN   
             4-Bet+              0     NaN          NaN         NaN       NaN   
             BB Walk / Fold      0     NaN          NaN         NaN       NaN   
             Uncategorised       0     NaN          NaN         NaN       NaN   

                             pfr_pct  three_bet_pct  call_open_pct  \
position_pre preflop_line                                            
SB           3-Bet             100.0          100.0            0.0   
BTN          Open Raise        100.0            0.0            0.8   
             Limp/Complete       0.0            0.0            0.0   
             BTN Folded          0.0            0.0            0.0   
SB           ISO Raise         100.0            0.0            0.0   
             Limp/Complete       0.0            0.0            0.0   
             Call vs Open        0.0            0.0          100.0   
             BTN Folded          0.0            0.0            0.0   
BTN          ISO Raise           NaN            NaN            NaN   
             3-Bet               NaN            NaN            NaN   
             4-Bet+              NaN            NaN            NaN   
             Call vs Open        NaN            NaN            NaN   
             BB Walk / Fold      NaN            NaN            NaN   
             Uncategorised       NaN            NaN            NaN   
SB           Open Raise          NaN            NaN            NaN   
             4-Bet+              NaN            NaN            NaN   
             BB Walk / Fold      NaN            NaN            NaN   
             Uncategorised       NaN            NaN            NaN   

                             showdown_pct  went_to_flop  went_to_turn  \
position_pre preflop_line                                               
SB           3-Bet                    0.0          39.1          17.4   
BTN          Open Raise               0.0          52.3          31.8   
             Limp/Complete            0.0          83.7          51.2   
             BTN Folded               0.0           0.0           0.0   
SB           ISO Raise                0.0          75.0          62.5   
             Limp/Complete            0.0         100.0         100.0   
             Call vs Open             0.0         100.0          71.2   
             BTN Folded               0.0          13.4           8.1   
BTN          ISO Raise                NaN           NaN           NaN   
             3-Bet                    NaN           NaN           NaN   
             4-Bet+                   NaN           NaN           NaN   
             Call vs Open             NaN           NaN   

/tmp/ipykernel_2260/3257466663.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frame.groupby(list(group_cols), dropna=False)


hands  net_bb  net_dollars  bb_per_100  \
preflop_line   deepest_street                                           
Open Raise     Preflop            63    40.0         4.00       63.49   
               Flop               27    36.3         3.63      134.44   
3-Bet          River               3    34.1         3.41     1136.67   
               Preflop            14    27.2         2.72      194.29   
Open Raise     Turn               15    17.3         1.73      115.33   
3-Bet          Flop                5    12.9         1.29      258.00   
Limp/Complete  Flop               14    12.5         1.25       89.29   
3-Bet          Turn                1     5.4         0.54      540.00   
ISO Raise      Flop                2     5.4         0.54      270.00   
Limp/Complete  Turn                9     5.1         0.51       56.67   
ISO Raise      Turn                1     1.8         0.18      180.00   
Call vs Open   Turn               11    -0.7        -0.07       -6.36   
ISO Raise      Preflop             4    -1.0        -0.10      -25.00   
BTN Folded     Flop                9    -5.2        -0.52      -57.78   
Limp/Complete  Preflop             7    -7.0        -0.70     -100.00   
BTN Folded     Turn                5   -11.0        -1.10     -220.00   
Call vs Open   Flop               15   -23.8        -2.38     -158.67   
               River              26   -32.6        -3.26     -125.38   
BTN Folded     River               9   -36.3        -3.63     -403.33   
               Preflop           235   -43.5        -4.35      -18.51   
ISO Raise      River               9   -45.0        -4.50     -500.00   
Open Raise     River              27   -55.5        -5.55     -205.56   
Limp/Complete  River              16   -71.8        -7.18     -448.75   
4-Bet+         Preflop             0     NaN          NaN         NaN   
               Flop                0     NaN          NaN         NaN   
               Turn                0     NaN          NaN         NaN   
               River               0     NaN          NaN         NaN   
Call vs Open   Preflop             0     NaN          NaN         NaN   
BB Walk / Fold Preflop             0     NaN          NaN         NaN   
               Flop                0     NaN          NaN         NaN   
               Turn                0     NaN          NaN         NaN   
               River               0     NaN          NaN         NaN   
Uncategorised  Preflop             0     NaN          NaN         NaN   
               Flop                0     NaN          NaN         NaN   
               Turn                0     NaN          NaN         NaN   
               River               0     NaN          NaN         NaN   

                               vpip_pct  pfr_pct  three_bet_pct  \
preflop_line   deepest_street                                     
Open Raise     Preflop            100.0    100.0            0.0   
               Flop               100.0    100.0            0.0   
3-Bet          River              100.0    100.0          100.0   
               Preflop            100.0    100.0          100.0   
Open Raise     Turn               100.0    100.0            0.0   
3-Bet          Flop               100.0    100.0          100.0   
Limp/Complete  Flop               100.0      0.0            0.0   
3-Bet          Turn               100.0    100.0          100.0   
ISO Raise      Flop               100.0    100.0            0.0   
Limp/Complete  Turn               100.0      0.0            0.0   
ISO Raise      Turn               100.0    100.0            0.0   
Call vs Open   Turn               100.0      0.0            0.0   
ISO Raise      Preflop            100.0    100.0            0.0   
BTN Folded     Flop                 0.0      0.0            0.0   
Limp/Complete  Preflop            100.0      0.0            0.0   
BTN Folded     Turn                 0.0      0.0            0.0   
Call vs Open   Flop               100.0      0.0  

## Next Analysis Ideas

- Slice results by stake and local session to spot when the heads-up lobby is softest.
- Compare villain action frequencies (fold/call/3-bet) by preflop line using `actions` joins.
- Add flop texture buckets (paired, monotone, high-card) to understand post-flop leaks.
- Track stack depth in big blinds to separate deep ante battles from short-stack push/fold spots.
- Build quick visualisations (line charts, heatmaps) once the aggregates stabilise.
- Extend to near-heads-up (3 players) to contrast adjustments as another player sits in.